# Obtenção da proporção de casos hospitalares por UF

## Introdução

Um parâmetro fundamental para modelar uma epidemia é determinar a taxa de transmissão por indivíduo infectado e por unidade de tempo. Nos modelos original e modificado, essa taxa é denominada $\beta$ e subdividida em diferentes "*betas*" ($\beta_1$, $\beta_2$...), correspondentes a cada compartimento em que os indivíduos sejam transmissíveis.

Os valores assumidos por esses diferentes *betas* - que, inclusive, podem mudar no tempo, de acordo com as medidas de contenção adotadas - têm impacto direto na velocidade com que novos indivíduos suscetíveis serão expostos ao agente infeccioso. 

Na implementação original proposta para o SimulaCovid (modelo SEIR), esses *betas* eram calculados assumindo que a transmissão dos indivíduos infectados com sintomas moderados correspondia a 90% da transmissão efetiva, enquanto as taxas de transmissão dos indivíduos com sintomas graves e críticos eram iguais entre si e correspondiam, somadas, aos outros 10% da taxa de transmissão efetiva.

Na implementação alternativa aqui proposta (modelo SEAPMDR), o cálculo deve incorporar novas taxas de transmissão, correspondentes aos indivíduos pré-sintomáticos transmissíveis ($\beta_E$) e assintomáticos ($\beta_0$). De maneira similar ao que sugere [Hill (2020)](https://alhill.shinyapps.io/COVID19seir/), assumimos que os casos pré-sintomáticos, assintomáticos e sintomáticos moderados têm a mesma transmissibilidade. **Já os casos sintomáticos graves e críticos são calculados à parte, considerando que sejam os únicos responsáveis pela fração de casos cuja transmissão ocorre em hospitais (nosocomiais).**

## Fonte dos dados

Para calcular os *betas*, portanto, é importante conhecer a proporção dos casos nosocomiais no total de eventos de transmissão.

No Brasil, os casos de Síndrome Respiratória Aguda Grave - SRAG (que incluem os casos mais severos de COVID-19) são notificados por meio de um sistema de vigilância epidemiológico específico do Ministério da Saúde. Os dados estão disponíveis [publicamente][Dados SRAG] e incluem a informação de sobre o caso ter ou não origem nosocomial.

Utilizamos esse dado da vigilância da SRAG e extrapolamos para todo o conjunto das infecções por COVID-19, assumindo que a proporção de casos nosocomiais entre os casos de SRAG associáveis à COVID-19 seja idêntico à proporção nas formas mais leves da doença. 

[Dados SRAG]: https://opendatasus.saude.gov.br/dataset/bd-srag-2020

In [7]:
import pandas as pd

In [8]:
srag_url = "https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/2020/INFLUD-25-01-2021.csv"
df_original = pd.read_csv(srag_url, sep=";", quotechar='"')

/mnt/c/Users/bernardo.baron/Projetos/impulsogov-desafio-simulacovid/.venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (15,18,54,60,62,63,64,92,94,106,108,115,117,118,119,123,144) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
df = df_original.copy()
df

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,DS_AN_OUT,TP_AM_SOR,SOR_OUT,DT_CO_SOR,TP_SOR,OUT_SOR,DT_RES,RES_IGG,RES_IGM,RES_IGA
0,15/01/2020,3,13/01/2020,3,RJ,NaN,NaN,RIO DE JANEIRO,330455,MS HSE HOSPITAL DOS SERVIDORES DO ESTADO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18/02/2020,8,15/02/2020,7,RS,002 CRS,1607.0,PORTO ALEGRE,431490,HOSPITAL NOSSA SENHORA DA CONCEICAO SA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28/02/2020,9,25/02/2020,9,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,HOSPITAL UNIVERSITARIO EVANGELICO MACKENZIE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27/02/2020,9,24/02/2020,9,MG,VARGINHA,1466.0,ITAMONTE,313300,CASA DE CARIDADE DE ITAMONTE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26/02/2020,9,25/02/2020,9,SP,GVE XXVIII CARAGUATATUBA,1576.0,SAO SEBASTIAO,355070,HOSPITAL DE CLINICAS DE SAO SEBASTIAO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163438,15/01/2021,2,01/01/2021,53,RJ,NaN,NaN,RIO DE JANEIRO,330455,INI FIOCRUZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1163439,30/11/2020,49,15/11/2020,47,RJ,NaN,NaN,RIO DE JANEIRO,330455,SES RJ HOSPITAL ESTADUAL CARLOS CHAGAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1163440,07/01/2021,1,31/12/2020,53,SP,GVE XVIII FRANCA,1343.0,ORLANDIA,353430,HOSPITAL BENEFICENTE SANTO ANTONIO ORLANDIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1163441,18/12/2020,51,12/12/2020,50,RJ,NaN,NaN,CAMPOS DOS GOYTACAZES,330100,HOSPITAL DR BEDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Processamento dos dados

Para obter a proporção de casos nosocomiais, o conjunto de dados da vigilância da SRAG obtido do Ministério da Saúde precisa passar por um pré-processamento:

1. Filtrar a coluna de "*Classificação final do caso*" (`CLASSI_FIN`) para obter apenas os casos confirmados de COVID-19, desprezando os casos de SRAG causados por outros agentes (valor "**5-SRAG por COVID-19**").
2. Filtrar a coluna "*Trata-se de caso nosocomial  (infecção adquirida  no hospital)?*" (`NOSOCOMIAL`) para garantir que está preenchida com um valor "**1-Sim**" ou "**2-Não**" (e não "**9-Ignorado**").
3. Transformar a coluna `NOSOCOMIAL` para receber valores binários - isto é, com valores "**0-Não nosocomial**" e "**1-Sim, é nosocomial**".
4. Manter apenas as colunas desejadas - sigla da Unidade Federativa (`SG_UF_NOT`), data da notificação (`DT_NOTIFIC` - apenas para fins informativos; não é utilizada nos cálculos posteriores) e `NOSOCOMIAL`.

Para mais detalhes sobre o significado das colunas e dos valores, consulte o [dicionário de dados] da base.

[dicionário de dados]: https://opendatasus.saude.gov.br/dataset/ae90fa8f-3e94-467e-a33f-94adbb66edf8/resource/8f571374-c555-4ec0-8e44-00b1e8b11c25/download/dicionario-de-dados-srag-hospitalizado-27.07.2020-final.pdf

In [10]:
# select only confirmed COVID-19 cases
df = df.loc[df["CLASSI_FIN"]==5, :]

# drop records with invalid values the target column
df = df.loc[df["NOSOCOMIAL"]<9, :]

# make target colum binary
df["NOSOCOMIAL"] = 2 - df["NOSOCOMIAL"]

# select only the columns needed
df = df[["SG_UF_NOT", "DT_NOTIFIC", "NOSOCOMIAL"]]

# convert date column
df["DT_NOTIFIC"] = pd.to_datetime(df["DT_NOTIFIC"], format='%d/%m/%Y')

df

,SG_UF_NOT,DT_NOTIFIC,NOSOCOMIAL
11,SP,2020-03-25,0.0
20,SP,2020-03-19,0.0
51,SP,2020-03-20,0.0
52,SP,2020-03-25,0.0
61,SP,2020-03-21,0.0
...,...,...,...
1163427,SC,2021-01-20,0.0
1163430,PR,2021-01-08,0.0
1163434,DF,2021-01-09,0.0
1163436,SP,2021-01-04,0.0


## Cálculo da proporção de casos nosocomiais

Com os dados pré-processados, os registros são agrupados por Unidade Federativa. São contados o total de casos (coluna final `total_srag`) e somados os casos nosocomiais (coluna `nosocomial_srag`). 

A divisão do segundo pelo primeiro corresponde à proporção de casos de Síndrome Respiratória Grave associados à COVID-19 associados à transmissão hospitalar em cada Unidade da Federação, desde o início da pandemia.

In [11]:
# get total number of cases and sum of nosocomial ones
nosocomial = df.groupby("SG_UF_NOT").agg(["sum", "count"])
nosocomial.columns = nosocomial.columns.get_level_values(1)

# rename columns
nosocomial.rename(columns = {"sum":"nosocomial_srag", "count": "total_srag"}, inplace=True)
nosocomial.index.rename("state_id", inplace=True)

# calculate the proportion
nosocomial["nosocomial_proportion"] = nosocomial["nosocomial_srag"] / nosocomial["total_srag"]

nosocomial

,nosocomial_srag,total_srag,nosocomial_proportion
state_id,,,
AC,0.0,462,0.000000
AL,143.0,3979,0.035939
AM,278.0,13461,0.020652
AP,18.0,2118,0.008499
BA,740.0,16698,0.044317
CE,521.0,16996,0.030654
DF,487.0,11916,0.040869
ES,174.0,4297,0.040493
GO,269.0,14207,0.018934


In [12]:
# save
nosocomial.to_csv("../data/nosocomial_srag.csv")